In [1]:
from datetime import datetime, timedelta
import time
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
keywords = ['股利', '營業收入', '第一季', '第1季', '貢獻', 'EPS', '打入', '供應鏈', '跨足', '訂單', '採用', '新藥', '認證', '通過', 'FDA', '簽訂', '合作', '自結', '結盟', '補助', '資產', '售地', '判賠', '訴訟', '官司', '記者會', '合約', '疫情', '新冠', '現金增資', '取得', '有價證券', '收購', '併購', '土地', '業外']
seasons = ['第一季', '第二季', '第三季', '第四季', '第1季', '第2季', '第3季', '第4季']
# '政府', '行政院'
lst_keywords = list(set(keywords + seasons))

# 每次程式執行中間休息時間
wait_minutes = 10

# line notify token
token = '8tia3RLmEzqUCqZ0KbkH2mJGosy484K6bNn9oDSKxAI'   # 工商時報通知 - test群組

def get_ctee_news(begin_time, lst_keywords):
    firefox_options = Options()
    firefox_options.add_argument("--headless")  # 無頭模式，不打開瀏覽器窗口
    driver = webdriver.Firefox(options=firefox_options)
    # driver = webdriver.Firefox()

    # 打开網頁
    url = "https://www.ctee.com.tw/livenews"
    driver.get(url)

    # 等待頁面加載完成
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='newslist__card']")))
    news_lst = driver.find_elements(By.XPATH, "//div[@class='newslist__card']") # 找到新聞位置

    # 取出在指定時間中的新聞
    df_news_ctee = pd.DataFrame()
    for i in range(len(news_lst)):
        title = news_lst[i].find_element(By.XPATH, ".//h3[@class='news-title']").text
        date = news_lst[i].find_element(By.CLASS_NAME, "news-date").text.replace('.', '/')
        date_time = news_lst[i].find_element(By.CLASS_NAME, "news-time").text
        link = news_lst[i].find_element(By.XPATH, ".//h3[@class='news-title']").find_element(By.XPATH, "./a").get_attribute("href")
        if date_time >= begin_time:
    #         print('============================')
    #         print('標題:', title)
    #         print('日期:', date)
    #         print('時間:', date_time)
    #         print('網址:', link)
            if any(keyword in title for keyword in lst_keywords):
                df_news_ctee = pd.concat([df_news_ctee, pd.DataFrame({
                    '標題': [title],
                    '日期': [date],
                    '時間': [date_time],
                    '網址': [link],
                })])
        else:
            break
    #         print('============================')
    #         print('標題:', title)
    #         print('時間:', date_time)
    df_news_ctee.reset_index(drop=True, inplace=True)
    driver.close()
    driver.quit()   # 關閉
    
    return df_news_ctee

def lineNotifyMessage(token, msg):
    headers = {
        "Authorization": 'Bearer ' + token, 
        # "Content-Type" : "application/x-www-form-urlencoded"
    }

    data = {'message': msg}
    
    try:
        r = requests.post("https://notify-api.line.me/api/notify", headers=headers, data=data)
        r.raise_for_status()  # 如果發生錯誤，會丟出相應的異常
        return r.status_code
    except requests.exceptions.RequestException as e:
        print("Error:", e)
        return None

# ============================================================
# main func
for i in range(50):
    if i == 0:
        begin_time = '00:01'
    else:
        # (datetime.now() - timedelta(minutes=5)).strftime("%Y-%m-%d %H:%M:%S") 
        begin_time = (datetime.now() - timedelta(minutes=wait_minutes)).strftime("%H:%M")
    
    # -------------
    print(begin_time, '~', datetime.now().strftime("%H:%M"), '開始執行')
    df_news_ctee = get_ctee_news(begin_time, lst_keywords)
    
    # if not df_news_ctee.empty:
        # df_news_ctee = df_news_ctee[df_news_ctee['標題'].str.contains('|'.join(lst_keywords))]
    if not df_news_ctee.empty:
        for i in df_news_ctee.index:
            message = (
                # '工商時報' + '\n' + 
                '\n' + 
                '📝' + df_news_ctee.at[i, '標題'] + '\n' + 
                '🔗網址:' + '\n' + 
                df_news_ctee.at[i, '網址']
            )
            # print(message)
            lineNotifyMessage(token, message)
        print(begin_time, '~', datetime.now().strftime("%H:%M"), 'done')
    else:
        print(begin_time, '~', datetime.now().strftime("%H:%M"), 'no news')
    
    time.sleep(wait_minutes*60)

00:01 ~ 08:26 開始執行
00:01 ~ 08:26 done
08:26 ~ 08:36 開始執行
08:26 ~ 08:37 no news
08:37 ~ 08:47 開始執行
08:37 ~ 08:47 no news
08:47 ~ 08:57 開始執行
08:47 ~ 08:57 done
08:57 ~ 09:07 開始執行
08:57 ~ 09:07 no news
09:07 ~ 09:17 開始執行
09:07 ~ 09:18 no news
09:18 ~ 09:28 開始執行
09:18 ~ 09:28 no news
09:28 ~ 09:38 開始執行
09:28 ~ 09:38 no news
09:38 ~ 09:48 開始執行
09:38 ~ 09:49 no news
09:49 ~ 09:59 開始執行
09:49 ~ 09:59 no news
09:59 ~ 10:09 開始執行
09:59 ~ 10:09 no news
10:09 ~ 10:19 開始執行
10:09 ~ 10:22 no news
10:22 ~ 10:32 開始執行
10:22 ~ 10:33 no news
10:33 ~ 10:43 開始執行
10:33 ~ 10:43 no news
10:43 ~ 10:53 開始執行
10:43 ~ 10:53 no news
10:53 ~ 11:03 開始執行
10:53 ~ 11:03 done
11:03 ~ 11:13 開始執行
11:03 ~ 11:14 no news
11:14 ~ 11:24 開始執行
11:14 ~ 11:27 no news
11:27 ~ 11:37 開始執行
11:27 ~ 11:37 no news
11:37 ~ 11:47 開始執行
11:37 ~ 11:48 no news
11:48 ~ 11:58 開始執行
11:48 ~ 11:58 no news
11:58 ~ 12:08 開始執行
11:58 ~ 12:08 no news
12:08 ~ 12:18 開始執行
12:08 ~ 12:18 no news
12:18 ~ 12:28 開始執行
12:18 ~ 12:29 no news
12:29 ~ 12:39 開始執行
12:29 

In [4]:
df_news_ctee

""


In [ ]:
begin_time = '07:00'
df_news_ctee = get_ctee_news(begin_time)
df_news_ctee

In [ ]:
df_news_ctee = df_news_ctee[df_news_ctee['標題'].str.contains('|'.join(lst_keywords))].reset_index(drop=True)
df_news_ctee

# request

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 工商時報
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36", 
#     "Referer": "https://www.google.com/"  #參照位址
# }
url="https://www.ctee.com.tw/livenews/policy"
headers={
    'Referer': 'https://www.google.com/',
    'content-type' : 'text/html; charset=UTF-8',
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)
response

In [ ]:
soup = BeautifulSoup(response.text,"lxml")
soup

In [ ]:
i = 0
df_ctee = pd.DataFrame()
for i in range(20):
    print('news od', i, '=======')
    # 標題
    title = soup.select("div.newslist__card")[i].select("h3")[0].text
    print(title)
    # 時間 (日期+時間)
    date_time = soup.select("div.newslist__card")[i].select("time")[0].text + ' ' + soup.select("div.newslist__card")[i].select("time")[1].text
    print(date_time)
    # 新聞網址
    href = soup.select("div.newslist__card")[i].select("h3")[0].a.attrs['href'] 
    href = 'https://www.ctee.com.tw'+href
    print(href)
    
    temp = pd.DataFrame({
        'title':[title],
        '時間':[date_time],
        'url':[href]
    })
    
    df_ctee = pd.concat([df_ctee,temp],axis=0)

df_ctee = df_ctee.reset_index(drop=True)
df_ctee

# line notify